# Predicting where a crime will occur/ or if there is a gun involved

Marissa Berk

In [13]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #We need this to split the data

In [14]:
df = pd.read_csv('KCPD_Crime_Data_2020.csv')
df.head(30) #let's take a look at the dataset

,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,...,Zip Code,Rep_Dist,Area,DVFlag,Involvement,Race,Sex,Age,Firearm Used Flag,Location
0,KC20000204,01/01/2020,18:39,01/01/2020,18:39,NaN,NaN,Robbery (Residential),120,Robbery,...,64128.0,PJ3113,EPD,N,VIC WIT,B,M,22.0,True,NaN
1,KC20000380,01/02/2020,14:00,12/30/2019,17:50,12/30/2019,18:00,Stealing – Shoplift,23C,Shoplifting,...,NaN,NaN,CPD,N,VIC,NaN,NaN,NaN,False,NaN
2,KC20000558,01/03/2020,08:33,01/03/2020,08:30,NaN,NaN,Domestic Violence Assault (Aggravated),13A,Aggravated Assault,...,64155.0,NaN,SCP,Y,VIC,W,M,50.0,False,NaN
3,KC20001168,01/05/2020,19:00,01/05/2020,18:40,NaN,NaN,Stealing – Other,23H,All Other Larceny,...,NaN,NaN,MPD,N,VIC,W,M,21.0,False,NaN
4,KC20001220,01/06/2020,00:43,01/06/2020,00:43,NaN,NaN,Robbery (Armed Street),120,Robbery,...,NaN,NaN,CPD,N,VIC,B,F,26.0,True,NaN
5,KC20001292,01/06/2020,11:03,01/06/2020,11:00,01/06/2020,13:30,Robbery (Residential),120,Robbery,...,64129.0,NaN,MPD,N,SUS,B,M,NaN,True,NaN
6,KC20001372,01/06/2020,16:36,01/06/2020,08:00,01/06/2020,16:00,Loss,NaN,NaN,...,64123.0,PJ0154,EPD,N,VIC,U,M,19.0,False,NaN
7,KC20001452,01/06/2020,20:54,01/05/2019,23:00,01/06/2019,06:00,Stolen Auto,240,Motor Vehicle Theft,...,NaN,NaN,EPD,N,SUS,W,M,NaN,False,NaN
8,KC20001566,01/07/2020,06:31,01/07/2020,08:58,NaN,NaN,Property Damage,290,Vandalism/Destruction of Property,...,64106.0,PJ1087,CPD,N,SUS,B,M,35.0,False,NaN
9,KC20001627,01/07/2020,13:34,01/07/2020,15:29,NaN,NaN,Assault (Non-Aggravated),13B,Simple Assault,...,64131.0,NaN,SPD,N,VIC,B,F,46.0,False,NaN


The variable _area_ shows which division the offense occured in. The areas are all in Kansas City and consist of: Central, East, Metro, South, North, and Shoal Creek. We will be making a prediction model that can predict if a crime will occur in that area.

# Data Cleaning & Variable Selection

First, we have to make dummie variables for the variable are to make them numerical so we can see them in the correlation table.

In [15]:
#dummies = pd.get_dummies(df['Area'])#make dummie variables for the gender
#dummies
#df = pd.concat([df, dummies], axis=1) #the axis=1 means: add it to the columns
#df.head(10)

In [16]:
#dummies = pd.get_dummies(df['DVFlag'])#make dummie variables for the gender
#dummies
#df = pd.concat([df, dummies], axis=1) #the axis=1 means: add it to the columns
#df.head(10)

In [17]:
#dummies = pd.get_dummies(df['Sex'])#make dummie variables for the gender
#dummies
#df = pd.concat([df, dummies], axis=1) #the axis=1 means: add it to the columns
#df.head(10)

Now let's take a look at how the variables correlate to eachother so we can select which variables to use in our prediction model.

In [20]:
#df.corr()

In [24]:
dummies = pd.get_dummies(df['Firearm Used Flag'])#make dummie variables for the gender
dummies
df = pd.concat([df, dummies], axis=1) #the axis=1 means: add it to the columns
df.head(10)

KeyError: 'Firearm Used Flag'

In [23]:
df = df[['Reported_Time','From_Time', 'Offense', 'Description', 'Area','Involvement','Sex','Age']] #make this selection before you dropna
df = df.dropna() #first get rid of rows with empty cells 
df['Firearm Used Flag'].value_counts()

KeyError: 'Firearm Used Flag'